In [1]:
!nvidia-smi

Tue Nov 14 21:35:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:86:00.0 Off |                  Off |
| N/A   52C    P0   148W / 150W |  32050MiB / 32768MiB |     94%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig
)

In [3]:
from datasets import Dataset, load_dataset
import torch
import random
import pandas as pd

In [4]:
model_name = "mistralai/Mistral-7B-v0.1"
device="cuda:0"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(f'../tokenizers/{model_name}')

In [9]:
model = AutoModelForCausalLM.from_pretrained(f'../models/{model_name}')

2023-11-14 21:11:45.202281: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-14 21:11:45.203963: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 21:11:45.240211: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 21:11:46.029779: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

In [11]:
df = pd.read_csv('ngs.csv')
df.drop(labels=['Unnamed: 0'], inplace=True, axis='columns')

In [12]:
def parse_answer(text):
    answer = text.split("A:")[-1].strip()
    return answer

def construct_prompt(train_dataset, num_demonstrations):
    assert num_demonstrations > 0
    prompt = ''
    train_examples = train_dataset.shuffle().select(range(num_demonstrations))
    for exemplar_num in range(num_demonstrations):
        train_example = train_examples[exemplar_num]
        use_bad_sentence = random.choice([True, False])
        exemplar = "Q: Is this sentence grammatical? Yes or No: "
        if use_bad_sentence:
            exemplar += train_example["ng-" + col]
            exemplar += "\nA: No"
        else:
            exemplar += train_example[col]
            exemplar += "\nA: Yes"
        exemplar += "\n\n"
        prompt += exemplar
    return prompt

def compute_accuracy(preds, golds):
    assert len(preds) == len(golds)
    total = 0
    correct = 0
    for pred, gold in zip(preds, golds):
        if pred == gold:
            correct += 1
        total += 1
    return correct / total

In [ ]:
preds = []
golds = []

filename = "classification-train-test.txt"
f = open(filename, "a")

print("____________________CLASSIFICATION (TRAIN TEST)____________________\n")
f.write("____________________CLASSIFICATION (TRAIN TEST)____________________\n")

gCols = [col for col in df.columns if not 'ng' in col]
datasets = {}
for col in gCols:
    datasets[col] = Dataset.from_pandas(pd.DataFrame(df[[col, 'ng-' + col]].copy())).train_test_split(test_size=0.2)

master_prompt = 'We will provide you a set of sentences which follow or violate a grammatical structure. \n The sentences may use subjects and objects from the following nouns - author, banana, biscuit, book, bottle, box, boy, bulb, cap, cat, chalk, chapter, cucumber, cup, dog, fish, fruit, girl, Gomu, Harry, hill, John, Leela, man, Maria, meal, mountain, mouse, newspaper, pear, pizza, poem, poet, rock, roof, Sheela, speaker, staircase, story, teacher, Tom, toy, tree, woman, writer.\nThe sentences may use any of the following verbs - brings, carries, claims, climbs, eats, holds, notices, reads, says, sees, states, takes.\n Each noun in a sentence may sometimes use a different determiner than those found in English. Here is a reference of determiners that can be used by nouns: "pear": "kar", "author": "kon", "authors": "kons", "banana": "kar", "biscuit": "kon", "book": "kon", "bottle": "kar", "box": "kar", "boy": "kon", "boys": "kons", "bulb": "kar", "cabinet": "kar", "cap": "kon", "cat": "kon", "cats": "kons", "chapter": "kon", "chalk": "kon", "cup": "kar", "cucumber": "kon", "dog": "kon", "dogs": "kons", "fish": "kon", "fruit": "kar", "girl": "kar", "girls": "kars", "hill": "kar", "man": "kon", "men": "kons", "meal": "kon", "mountain": "kar", "mouse": "kon", "newspaper": "kon", "pizza": "kar", "poet": "kon", "poets": "kons", "poem": "kar", "rock": "kon", "roof": "kon", "speaker": "kon", "speakers": "kons", "staircase": "kar", "story": "kar", "teacher": "kon", "teachers": "kons", "toy": "kon", "tree": "kar", "woman": "kar", "women": "kars", "writer": "kon", "writers": "kons". Each verb in a sentence may sometimes use the past tense of the verb if it is more appropriate. Here are a set of verbs and their past tenses - "climbs" : "climbed", "reads": "read", "carries": "carried", "eats": "ate", "holds": "held", "takes" :"took", "brings": "brought", "reads": "read", "climb" : "climbed", "read": "read", "carry": "carried", "eat": "ate", "hold": "held", "take" :"took", "bring": "brought", "read": "read"\n The sentences may sometimes use the infinitive forms of a verb. Here are a set of verbs and their infinitives - "climbs" : "to climb", "reads": "to read", "carries": "to carry", "eats": "to eat", "holds": "to hold", "takes" : "to take", "brings": "to bring", "reads": "to read", "climb" : "to climb", "read": "to read", "carry": "to carry", "eat": "to eat", "hold": "to hold", "take" : "to take", "bring": "to bring", "read": "to read". \n The sentences may sometimes use the plural form of a noun. Here are a set of nouns and their plurals - "fish": "fish", "mouse": "mice", "bottle": "bottles", "newspaper": "newspapers", "chalk": "chalks", "box": "boxes", "cap": "caps", "bulb": "bulbs", "cup": "cups", "toy": "toys", "staircase": "staircases", "rock": "rocks", "hill": "hills", "mountain": "mountains", "roof": "roofs", "tree": "trees", "biscuit": "biscuits", "banana": "bananas", "pear": "pears", "meal": "meals", "fruit": "fruits", "cucumber": "cucumbers", "pizza": "pizzas", "book": "books", "poem": "poems", "story": "stories", "chapter": "chapters". \n The sentences may sometimes use the passive form of a verb. Here are a set of verbs and their passive forms - "carries": "carried", "carry": "carried", "holds": "held", "hold": "held", "takes": "taken", "take": "taken", "brings": "brought", "bring": "brought", "climbs": "climbed", "climb": "climbed", "eats": "eaten", "eat": "eaten", "reads": "read", "read": "read"\n\n'

for NUM_DEMONSTRATIONS in range(10, 15, 5):
    print(f"____________________NUM DEMONSTRATIONS = {NUM_DEMONSTRATIONS}____________________\n")
    f.write(f"____________________NUM DEMONSTRATIONS = {NUM_DEMONSTRATIONS}____________________\n")
    f.flush()
    for col in gCols:
        filename2 = f"classification-train-test-{col}-prompts.txt"
        m = open(filename2, "a")
        train_dataset = datasets[col]['train']
        test_dataset = datasets[col]['test']
        prompt = ''
        printAnswer = True
        for test_sentence in train_dataset:
            testBadOrGood = random.choice(['ng-', ''])
            prompt = construct_prompt(train_dataset, NUM_DEMONSTRATIONS)
            # Append test example
            prompt += "Q: Is this sentence grammatical? Yes or No: "
            prompt += test_sentence[testBadOrGood + col]
            prompt += "\nA:"
            if testBadOrGood == 'ng-':
                golds.append("No")
            else:
                golds.append("Yes")

            # Get answer from model
            model_inputs = tokenizer([master_prompt + prompt], return_tensors="pt").to(device)
            # answer = model.generate(prompt_tok,
            #                     top_p=0.9, temperature=0.1,
            #                     max_new_tokens=2)
            answer = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=2, do_sample=True)
            answer = tokenizer.batch_decode(answer)[0]
            if printAnswer:
                print(answer)
                printAnswer = False
            m.write(answer.split(master_prompt)[1] + "\n\n-------------------------------------\n\n")
            m.flush()
            preds.append(parse_answer(answer))

        m.close()
        # Evaluate
        accuracy = compute_accuracy(preds, golds)
        print(f"{col} -- Accuracy: {accuracy:.2f}\n")
        f.write(f"{col} -- Accuracy: {accuracy:.2f}\n")
        f.flush()

____________________CLASSIFICATION (TRAIN TEST)____________________

____________________NUM DEMONSTRATIONS = 10____________________

<s> We will provide you a set of sentences which follow or violate a grammatical structure. 
 The sentences may use subjects and objects from the following nouns - author, banana, biscuit, book, bottle, box, boy, bulb, cap, cat, chalk, chapter, cucumber, cup, dog, fish, fruit, girl, Gomu, Harry, hill, John, Leela, man, Maria, meal, mountain, mouse, newspaper, pear, pizza, poem, poet, rock, roof, Sheela, speaker, staircase, story, teacher, Tom, toy, tree, woman, writer.
The sentences may use any of the following verbs - brings, carries, claims, climbs, eats, holds, notices, reads, says, sees, states, takes.
 Each noun in a sentence may sometimes use a different determiner than those found in English. Here is a reference of determiners that can be used by nouns: "pear": "kar", "author": "kon", "authors": "kons", "banana": "kar", "biscuit": "kon", "book": "

In [ ]:
preds = []
golds = []

gCols = [col for col in df.columns if not 'ng' in col]
datasets = {}
for col in gCols:
    datasets[col] = Dataset.from_pandas(pd.DataFrame(df[[col, 'ng-' + col]].copy())).train_test_split(test_size=0.2)

filename = "classification-train-train.txt"
f = open(filename, "a")


print("____________________CLASSIFICATION (TRAIN TRAIN)____________________\n")
f.write("____________________CLASSIFICATION (TRAIN TRAIN)____________________\n")

for NUM_DEMONSTRATIONS in range(10, 30, 5):
    print(f"____________________NUM DEMONSTRATIONS = {NUM_DEMONSTRATIONS}____________________\n")
    f.write(f"____________________NUM DEMONSTRATIONS = {NUM_DEMONSTRATIONS}____________________\n")
    for col in gCols:
        train_dataset = datasets[col]['train']
        test_dataset = datasets[col]['test']
        prompt = ''
        printAnswer = False
        for test_sentence in train_dataset:
            testBadOrGood = random.choice(['ng-', ''])
            prompt = construct_prompt(train_dataset, NUM_DEMONSTRATIONS)
            # Append test example
            prompt += "Q: Is this sentence grammatical? Yes or No: "
            prompt += test_sentence[testBadOrGood + col]
            prompt += "\nA:"
            if testBadOrGood == 'ng-':
                golds.append("No")
            else:
                golds.append("Yes")

            # Get answer from model
            model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
            # answer = model.generate(prompt_tok,
            #                     top_p=0.9, temperature=0.1,
            #                     max_new_tokens=2)
            answer = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=2, do_sample=True)
            answer = tokenizer.batch_decode(answer)[0]
            if printAnswer:
                print(answer)
                printAnswer = False
            preds.append(parse_answer(answer))

        # Evaluate
        accuracy = compute_accuracy(preds, golds)
        print(f"{col} -- Accuracy: {accuracy:.2f}\n")
        f.write(f"{col} -- Accuracy: {accuracy:.2f}\n")